In [ ]:
#import
import scipy as sci
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Define a function to calculate walk speed based on age and disability
def get_walk_speed(age, disability):
    if age > 70 or age < 8:
        if disability == 1:
            mean_time = 1.2  # 20% increase for age > 70 or < 8 and disability
        else:
            mean_time = 1.2  # 20% increase for age > 70 or < 8 and no disability
    elif disability == 1:
        mean_time = 1.5  # 50% increase for passengers with disabilities
    else:
        mean_time = 1.0  # Default mean_time for other passengers

    stddev_time = 0.2
    walk_speed = np.random.normal(loc=mean_time, scale=stddev_time)
    return walk_speed

# Define a simulation function
def simulate_disembarking(disembarking_strategy):
    # Initialize Constants
    n_rows = 33
    n_cols = 6

    n_pass = n_rows * n_cols

    # Create seat matrix, using -1 to represent occupied seats
    seats = np.zeros((n_rows, n_cols))
    seats[:, :] = -1

    # Create aisle array
    aisle_q = np.zeros(n_rows)
    aisle_q[:] = -1

    # Create initial passenger number queue
    pass_q = [int(i) for i in range(n_pass)]
    pass_q = np.array(pass_q)

    # Define queue disobedience constant
    queue_disobedience = 0.1  # Adjust the value as needed

    # Calculate the extent of shuffling based on queue_disobedience
    shuffle_factor = int(queue_disobedience * n_pass)
    shuffle_factor = min(shuffle_factor, n_pass)

    # Shuffle the first 'shuffle_factor' passengers in the queue
    if shuffle_factor > 0:
        np.random.shuffle(pass_q[:shuffle_factor])

    # Account for passenger ages and physical passenger disabilities
    mean_age = 40
    stddev_age = 15
    passenger_ages = np.random.normal(mean_age, stddev_age, n_pass)
    passenger_ages = np.maximum(passenger_ages, 0)
    passenger_ages = np.round(passenger_ages)

    passenger_disabilities = np.random.choice([0, 1], n_pass, p=[0.95, 0.05])  # 5% chance of having a disability

    # Create arrays for seat numbers
    row_q_init = np.zeros(n_pass)
    col_q_init = np.zeros(n_pass)

    # Create arrays for passenger movement
    moveto_loc = np.zeros(n_pass)
    moveto_time = np.zeros(n_pass)
    moveto_loc_dict = {i: "aisle" for i in pass_q}  # All passengers initially in the aisle
    moveto_time_dict = {i: 0 for i in pass_q}

    # Create function to assign seat number to each passenger
    def AssignSeats(rq, cq, assign_type, n_pass=n_pass, n_rows=n_rows):
        if assign_type == "FrontToBack":
            i = 0
            f = n_rows
            c = [0, 5, 1, 4, 2, 3]
            count = 0

            while f <= n_pass:
                rq[i:f] = list(reversed(range(0, n_rows)))
                cq[i:f] = [c[count]] * n_rows
                i += n_rows
                f += n_rows
                count += 1

        elif assign_type == "BackToFront":
            av_rows = np.arange(0, n_rows, 1)
            av_rows = np.tile(av_rows, (n_cols, 1))
            av_rows = av_rows.T.flatten()
            av_cols = np.arange(0, n_cols, 1)
            av_cols = np.tile(av_cols, (n_rows, 1)).flatten()
            av_seats = np.zeros((n_pass, 2))

            for i in range(n_pass):
                av_seats[i] = [av_rows[i], av_cols[i]]

            # Reverse the order of passengers
            av_seats = av_seats[::-1]

            rq = av_seats[:, 0]
            cq = av_seats[:, 1]

        # ... (other seat assignment strategies)

        return rq, cq

    # Assign seating order based on the chosen strategy
    if disembarking_strategy == "FrontToBack":
        row_q, col_q = AssignSeats(row_q_init, col_q_init, "FrontToBack")
    elif disembarking_strategy == "BackToFront":
        row_q, col_q = AssignSeats(row_q_init, col_q_init, "BackToFront")

    # Create array for times
    time_q = [get_walk_speed(age, disability) for age, disability in zip(passenger_ages, passenger_disabilities)]

    # Create dictionaries for passenger information
    pass_dict = {}
    time_dict = {}
    seat_nos = np.column_stack((row_q, col_q))
    for i in range(n_pass):
        pass_dict[i] = seat_nos[i]
    for i in range(n_pass):
        time_dict[i] = time_q[i]

    # Create sum time array
    sum_time = np.zeros(n_pass)
    for i in range(n_pass):
        sum_time[i] = sum(time_q[:i + 1])

    # Define initial conditions
    time = 0
    time_step = 0.1
    exit_sum = np.sum(pass_q)
    pass_sum = np.sum(seats)

    while pass_sum != exit_sum:
        for passg in aisle_q:
            if passg != -1:
                if moveto_time_dict[passg] == 0:
                    if time > moveto_time_dict[passg]:
                        # Passenger is in the aisle, move them toward the exit
                        current_row = int(pass_dict[passg][0])
                        current_col = int(pass_dict[passg][1])

                        if moveto_loc_dict[passg] == "aisle":
                            next_row = current_row
                            next_col = current_col

                            if current_col < 5:
                                next_col += 1
                            else:
                                next_col -= 1

                            if next_col >= 0 and next_col <= 5 and seats[next_row][next_col] == -1:
                                # Move to the next seat or aisle
                                moveto_loc_dict[passg] = "aisle" if next_col in [1, 4] else "seat"
                                moveto_time_dict[passg] = time + time_dict[passg]

        # Iterate through passengers in the order of seating
        for row in range(n_rows):
            for col in [1, 4, 2, 3, 0, 5]:  # Aisle, middle, window order
                if seats[row][col] != -1:
                    passg = seats[row][col]
                    if moveto_time_dict[passg] == 0:
                        if moveto_loc_dict[passg] == "aisle":
                            # Check if the seat or aisle next to the passenger is empty
                            next_row = row
                            next_col = col
                            if col in [0, 5]:
                                next_col = 1 if col == 0 else 4
                            else:
                                next_col = 0 if col == 1 else 5
                            if seats[next_row][next_col] == -1:
                                # Move to the next seat or aisle
                                moveto_loc_dict[passg] = "seat"
                                moveto_time_dict[passg] = time + time_dict[passg]

        # Iteration timekeeping
        time += time_step
        pass_sum = np.sum(seats)

        print("Current state:")
        print("Time:", time)
        print("Passenger queue:", pass_q)
        print("Aisle queue:", aisle_q)

    return time

In [ ]:
num_simulations = 100

disembarking_times = []

for _ in range(num_simulations):
    result = simulate_disembarking("FrontToBack")
    if result > 0:
        boarding_times.append(result)
    else:
        print("negative number!")

# Calculate the average time
average_time = sum(disembarking_times) / len(disembarking_times)

# Calculate percentiles
percentile_5 = np.percentile(boarding_times, 5)
percentile_95 = np.percentile(boarding_times, 95)

print("Average disembarking time for {} simulations: {:.2f}".format(len(disembarking_times), average_time))
print("5th percentile disembarking time: {:.2f}".format(percentile_5))
print("95th percentile disembarking time: {:.2f}".format(percentile_95))

# Create a histogram of boarding times
plt.hist(boarding_times, bins=20, color='blue', alpha=0.7)
plt.xlabel("Disembarking Time")
plt.ylabel("Frequency")
plt.title("Histogram of Disembarking Times")
plt.grid(True)
plt.show()